In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from data_objects.article import Article
from data_objects.legislation import Legislation
from data_objects.public_consultation import PublicConsultation
from data_objects.base import Base
from typing import Any
import functools
import requests
import re
from sqlalchemy import create_engine, select, union_all, func, delete, and_
from sqlalchemy.orm import Session, aliased
from typing import Callable
from bs4 import BeautifulSoup
import os
from configparser import ConfigParser
from datetime import datetime
config = ConfigParser()
config.read("config.ini")
from pathlib import Path
import csv

In [3]:
single_results_path = Path("single_legislations.txt")

single_results = single_results_path.read_text()
print(single_results.split("\n"))

['11', '12', '13', '14', '16', '19', '20', '23', '24', '25', '27', '29', '30', '33', '34', '36', '37', '38', '39', '43', '45', '46', '49', '54', '62', '63', '66', '67', '68', '69', '70', '71', '78', '83', '84', '90', '95', '100', '110', '111', '119', '120', '122', '124', '129', '131', '135', '136', '138', '139', '141', '143', '146', '147', '148', '154', '160', '168', '179', '187', '189', '201', '218', '219', '226', '227', '230', '232', '233', '236', '239', '241', '243', '247', '249', '254', '258', '263', '281', '301', '303', '309', '310', '312', '313', '314', '317', '319', '321', '323', '324', '325', '329', '354', '357', '362', '363', '372', '379', '382', '384', '386', '387', '391', '393', '394', '396', '399', '406', '412', '416', '430', '436', '441', '444', '458', '459', '478', '483', '484', '485', '486', '497', '499', '504', '505', '514', '519', '521', '529', '530', '532', '533', '537', '539', '544', '546', '548', '549', '550', '556', '558', '559', '562', '563', '569', '574', '579', 

In [4]:
engine = create_engine(config.get('DEFAULT', 'db_file'))

In [13]:
with Session(engine) as sess:
    stmt = select(Legislation).where(Legislation.id.in_(single_results.split("\n")))
    print(stmt)
    legObjs = sess.execute(stmt).scalars()

    with open('single_result_leg.csv','w',encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        for legObj in legObjs:
            # print(legObj.id)
            if legObj.final_legislation_id is None:
                print(legObj.id," has None")
                continue
            writer.writerow([legObj.id,
                             legObj.title,
                             legObj.scrap_url,
                             legObj.final_legislation_id,
                             legObj.final_legislation.title,
                             legObj.final_legislation.fek_number,
                             legObj.final_legislation.scrap_url])

SELECT "Legislation".id, "Legislation".title, "Legislation".ministry, "Legislation".date_posted, "Legislation".parliament_url, "Legislation".legislation_pdf_url, "Legislation".scrap_url, "Legislation".final_legislation_id, "Legislation".fek_number, "Legislation".legislation_type 
FROM "Legislation" 
WHERE "Legislation".id IN (__[POSTCOMPILE_id_1])
14  has None
136  has None
143  has None
160  has None
179  has None
232  has None
241  has None
394  has None
436  has None
441  has None
444  has None
478  has None
505  has None
537  has None
550  has None
569  has None
612  has None
780  has None
810  has None
851  has None
854  has None


In [8]:
with Session(engine) as sess:
    stmt = select(Legislation).where(Legislation.id==11)
    print(stmt)
    l = sess.execute(stmt).scalars().one()
    print(l)
    print(l.final_legislation.title)

SELECT "Legislation".id, "Legislation".title, "Legislation".ministry, "Legislation".date_posted, "Legislation".parliament_url, "Legislation".legislation_pdf_url, "Legislation".scrap_url, "Legislation".final_legislation_id, "Legislation".fek_number, "Legislation".legislation_type 
FROM "Legislation" 
WHERE "Legislation".id = :id_1
Legislation(id=11, title=Διαβούλευση για το σχέδιο νόμου υπό τον τίτλο: «Ενιαίο ρυθμιστικό πλαίσιο για την οργάνωση και λειτουργία του Ελληνικού Γεωργικού Οργανισμού-ΔΗΜΗΤΡΑ», ministry=Υπουργείο Αγροτικής Ανάπτυξης και Τροφίμων, date_posted=2023-02-13)
Ενιαίο ρυθμιστικό πλαίσιο για την οργάνωση και λειτουργία του Ελληνικού Γεωργικού Οργανισμού - ΔΗΜΗΤΡΑ, σύσταση και λειτουργία φορέα διαχείρισης Παραδοσιακού Ελαιώνα Άμφισσας και άλλες διατάξεις για την ενίσχυση της αγροτικής ανάπτυξης
